In [ ]:
%load_ext autoreload
%autoreload 2

from pprint import pprint

from read_excel import *
from compute_state import *
from collections import namedtuple
from custom_types import *
from prices.balanz_fci_price import *
from timeseries_analysis import *
from datetime import datetime
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot

In [ ]:
# Example usage:
file_path = 'private/movimientos.xlsx'  # Set the filename to 'ordenes.xlsx'
df = read_excel_to_dataframe(file_path)
print(df.loc[0])

In [ ]:
# Create a State tuple containing the Balance tuple
current_state = State(Fondos=tuple(), Cedears=tuple(), Bonos=tuple(), Corporativos=tuple())

states_over_time = {}

# Iterate through all rows of the DataFrame
for index, row in df[::-1].iterrows():
    
    # Call the update_state function for each row
    next_state = update_state(row, current_state)
    
    # Update the current state to the next state
    current_state = next_state

    states_over_time[row['Concertacion'].date()] = current_state

current_state

In [ ]:
fondos_over_time = extract_instrument_from_state(states_over_time, 'Cedears')

In [ ]:
fondos_over_time_df = dict_to_dataframe(fondos_over_time)
fondos_over_time_df

In [ ]:
fondos_over_time_df.index[0]

In [ ]:
# fondos_over_time_df_filtered = fondos_over_time_df.drop(['BCMMA', 'BCAHA', 'BCRFA', 'BRTA', 'BCACCA'], axis=1)
fondos_over_time_df_filtered = fondos_over_time_df
# fondos_over_time_df_filtered = fondos_over_time_df.INSTITUA
fondos_over_time_df_filtered

In [ ]:
FCIs_amounts = fill_missing_days(fondos_over_time_df_filtered)
FCIs_amounts

In [ ]:
FCIs_amounts.index[0]

In [ ]:
FCIs_prices = get_cedears_price_df(FCIs_amounts.columns, FCIs_amounts.index)
FCIs_prices

In [ ]:
FCIs_prices.index[0]

In [ ]:
FCIs_values = FCIs_amounts*FCIs_prices
FCIs_values

In [ ]:
FCIs_values.index[0]

In [ ]:
pd.options.plotting.backend = "plotly"

# Plot using Plotly Express
fig = FCIs_values.plot()
fig.update_layout(
    title="CEDEAR position value",
    xaxis_title="Date",
S    yaxis_title="Value [USD]"
)
fig.show()

In [ ]:
FCIs_values_dolar = FCIs_values

In [ ]:
FCIs_values_dolar.index[0]

In [ ]:
# Plot using Plotly Express
fig = FCIs_values_dolar.plot()
fig.update_layout(
    title="FCI position in dollars",
    xaxis_title="Date",
    yaxis_title="Value [USD]"
)
fig.show()

In [ ]:
FCIs_deposits = FCIs_amounts.diff().fillna(0)
fig = FCIs_deposits.plot()
fig.update_layout(
    title="FCI desposits/extractions",
    xaxis_title="Date",
    yaxis_title="Value [Units]"
)
fig.show()

In [ ]:
FCI_deposits_values = FCIs_deposits*FCIs_prices
fig = FCI_deposits_values.plot()
fig.update_layout(
    title="FCI desposits/extractions",
    xaxis_title="Date",
    yaxis_title="Value [ARS]"
)
fig.show()

In [ ]:
FCI_deposits_values_dolar = FCI_deposits_values
fig = FCI_deposits_values_dolar.plot()
fig.update_layout(
    title="FCI desposits/extractions",
    xaxis_title="Date",
    yaxis_title="Value [USD]"
)
fig.show()

In [ ]:
FCI_deposits_values_dolar_cumulative = FCI_deposits_values.cumsum()
fig = FCI_deposits_values_dolar_cumulative.plot()
fig.update_layout(
    title="FCI cumulative value",
    xaxis_title="Date",
    yaxis_title="Value [USD]"
)
fig.show()

In [ ]:
FCIs_yield = FCIs_values_dolar-FCI_deposits_values_dolar_cumulative
fig = FCIs_yield.plot()
fig.update_layout(
    title="FCI total balance",
    xaxis_title="Date",
    yaxis_title="Balance [USD]"
)
fig.show()

In [ ]:
FCIs_yield_percent = FCIs_yield.div(FCI_deposits_values_dolar_cumulative.iloc[:, 0], axis=0)*100
fig = FCIs_yield_percent.plot()
fig.show()

In [ ]:
FCIs_yield_percent.mean()

In [ ]:
FCI_deposits_values_dolar

In [ ]:
xirr_interest = calculate_interest(FCI_deposits_values_dolar, FCIs_values_dolar)
fig = xirr_interest.plot()
fig.update_yaxes(range=[-400, 400])  # Set y-axis limits
fig.show()

In [ ]:
fig = xirr_interest.plot()
fig.update_yaxes(range=[-100, 4000])  # Set y-axis limits
fig.show()

In [ ]:
df1 = xirr_interest
df2 = FCIs_yield_percent

# Assuming df1 and df2 are your DataFrames
fig = go.Figure()

# Add traces for each DataFrame
fig.add_trace(go.Scatter(x=df1.index, y=df1['INSTITUA'], mode='lines', name='XIRR interest'))
fig.add_trace(go.Scatter(x=df2.index, y=df2['INSTITUA'], mode='lines', name='Percentage interest'))

# Update layout and show the figure
fig.update_layout(
    title='Comparison of Two DataFrames',
    xaxis_title='Date',
    yaxis_title='INSTITUA',
    legend_title='DataFrames'
)
fig.update_yaxes(range=[-40, 60])  # Set y-axis limits
fig.show()